In [ ]:
#!/usr/bin/env python
import numpy as np
from math import ceil, floor
from numpy.linalg import det, inv, matrix_rank
from copy import copy
from cvxopt.glpk import ilp
from cvxopt import matrix
from itertools import permutations, combinations

dimensions = 4
max_minor = 3

def minor(arr, i=0):  #  ith row removed
    return arr[np.array(range(i)+range(i+1, arr.shape[0]))[:, np.newaxis],
               np.array(range(arr.shape[1]))]


def get_minors(A):
    return [det(minor(A,i)) for i in xrange(A.shape[0])]


def generate_vertices(cone, b):
    vertices = []
    for i in xrange(cone.shape[0]):
        vertices.append(inv(minor(cone,i)).dot(b[range(i)+range(i+1, b.shape[0])]))
    return vertices


def is_valid_solution(solution):
    return  any(x for x in solution)

#         print restriction
#         restriction = np.asarray(restriction)#np.ndarray(buffer=np.asarray(restriction)))#, shape=(1, dimensions)))
def bruteforce(results, restriction):
    print restriction
    restricted = np.append(hermit, np.matrix([np.asarray(restriction)]), 0) # np.ndarray(buffer=restriction, shape=(1, dimensions))
    minors = get_minors(restricted)
    if not all(minors):
        return
    abs_minors = [abs(m) for m in minors]
    min_abs_minor = min(abs_minors)
    max_abs_minor = max(abs_minors)

    c = restricted[-1]
    x = np.zeros(shape=(dimensions,1), dtype=float)
    x[-1] = float(dimensions)/float(max_minor)
    sigma = c.dot(x)  # ( b[n+1])
    f = copy(x)  # np.reshape(np.append(x, sigma), newshape=(dimensions + 1, 1))
    f[-1] = float(dimensions + 1)/float(max_minor)

    is_in_M = c.dot(f) > sigma
    target_f = matrix(-np.ones((dimensions + 1,1),dtype=float), tc='d')
    s = None
    if is_in_M:
        sigma = floor(sigma)
        while not s:
#                 print 'sigma:', sigma
            b = np.append(f, sigma)
            (status,s) = ilp(target_f, 
                 G=matrix(restricted, tc='d'),
                 h=matrix(b, tc='d'),
                 I=set(xrange(dimensions)),
                )

            sigma += 1
            if status != 'optimal':
#                     print status
                break
        sigma -= 1
    else:
        sigma = ceil(sigma)
        while not s:
#                 print 'sigma:', sigma
            b = np.append(f, sigma)
            (status,s) = ilp(target_f, 
                 G=matrix(-restricted, tc='d'),
                 h=matrix(-b, tc='d'),
                 I=set(xrange(dimensions)),
                )
            sigma -= 1
            if status != 'optimal' or abs(sigma) > 100:
#                     print status
                break
        sigma += 1
    if not s:
        return
    print 'sigma after cycle:', sigma
    print 'sol:', s
    
    b = np.append(f, sigma)
    max_b = max(b)
    vv = generate_vertices(restricted, b)
    
    width_list = list()
    for i,j in combinations(xrange(len(vv)), 2):
        target_f2 = vv[i] - vv[j]
        restrictions = np.concatenate((inv(minor(restricted, i).T), -inv(minor(restricted, j).T),
                                       np.diag(np.ones(dimensions).astype('d'))))
        
        (status,s) = ilp(matrix(target_f2), 
             G=matrix(-restrictions, tc='d'),
             h=matrix(np.ones(shape=(restrictions.shape[0],1), dtype=np.double), tc='d'),
             I=set(range(len(vv)-1)),
            )
        if status == 'optimal':
            width_c = abs(target_f2.dot(s)[0])
            if is_valid_solution(s):
                width_list.append(width_c)
            else:
                return
        else:
                return
    print 'The next result is going to be added: [0][0]'
    result_ = ( min(width_list), dimensions, int(min_abs_minor), int(max_abs_minor), max_b ) 
    print(result_)
    with open( 'res_new.csv', 'a' ) as f:
        f.write(';'.join([str(l) for l in result_]) + '\r\n')
    results.append(result_)
#     results.append( ( min(width_list), dimensions, int(min_m), int(max_m), max_b ) )
    
diag = np.array([1]*(dimensions)).astype('i')
diag[-1] = max_minor
hermit = np.diag(diag)
# print hermit


from multiprocessing import Process, Value, Array, Manager

manager = Manager()
results = manager.list()

for pre_last_row in permutations(xrange(dimensions), dimensions-1):
    hermit[dimensions-1][:-1] = list(pre_last_row)
    
    for restriction in permutations(xrange(-max_minor, max_minor + 1), dimensions):
        p = Process(target=bruteforce, args=(results, restriction))
        p.start()
        p.join(20)
        if p.is_alive():
            print "running... let's kill it..."
            # Terminate
            p.terminate()
            p.join()
        
        
#         bruteforce(restriction)

(-3, -2, -1, 0)
(-3, -2, -1, 1)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 2.0)
(-3, -2, -1, 2)
sigma after cycle: 3.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.62962963]]), 4, 2, 9, 3.0)
(-3, -2, -1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.77777778]]), 4, 2, 9, 4.0)
(-3, -2, 0, -1)
(-3, -2, 0, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 1.6666666666666667)
(-3, -2, 0, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 9, 2.0)
(-3, -2, 0, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00

(-3, 2, -1, 0)
(-3, 2, -1, 1)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 2.0)
(-3, 2, -1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.77777778]]), 4, 2, 9, 4.0)
(-3, 2, 0, -2)
(-3, 2, 0, -1)
(-3, 2, 0, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 1.6666666666666667)
(-3, 2, 0, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.77777778]]), 4, 2, 9, 4.0)
(-3, 2, 1, -2)
(-3, 2, 1, -1)
(-3, 2, 1, 0)
(-3, 2, 1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[  1.33226763e-15]]), 4, 2, 9, 4.0)
(-3, 2, 3, -2)
(-3, 2, 3,

(-2, 0, 1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.77777778]]), 4, 2, 6, 4.0)
(-2, 0, 2, -3)
(-2, 0, 2, -1)
(-2, 0, 2, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 6, 1.6666666666666667)
(-2, 0, 2, 3)
(-2, 0, 3, -3)
(-2, 0, 3, -1)
(-2, 0, 3, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 0, 6, 1.6666666666666667)
(-2, 0, 3, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 6, 2.0)
(-2, 1, -3, -1)
(-2, 1, -3, 0)
(-2, 1, -3, 2)
sigma after cycle: 5.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 13, 5.0)
(-2, 1, -3, 3)



The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 8, 4.0)
(-1, -2, 2, -3)
(-1, -2, 2, 0)
(-1, -2, 2, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 6, 1.6666666666666667)
(-1, -2, 2, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

(-1, -2, 3, -3)
(-1, -2, 3, 0)
(-1, -2, 3, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 0, 6, 1.6666666666666667)
(-1, -2, 3, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 7, 2.0)
(-1, 0, -3, -2)
(-1, 0, -3, 1)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 11, 4.0)
(-1, 0, -3, 2)
sigma after cycle: 5.0
sol: [ 

(0, 2, -2, 3)
(0, 2, -1, -3)
(0, 2, -1, -2)
(0, 2, -1, 1)
(0, 2, -1, 3)
(0, 2, 1, -3)
(0, 2, 1, -2)
(0, 2, 1, -1)
(0, 2, 1, 3)
(0, 2, 3, -3)
(0, 2, 3, -2)
(0, 2, 3, -1)
(0, 2, 3, 1)
(0, 3, -3, -2)
(0, 3, -3, -1)
(0, 3, -3, 1)
(0, 3, -3, 2)
(0, 3, -2, -3)
(0, 3, -2, -1)
(0, 3, -2, 1)
(0, 3, -2, 2)
(0, 3, -1, -3)
(0, 3, -1, -2)
(0, 3, -1, 1)
(0, 3, -1, 2)
(0, 3, 1, -3)
(0, 3, 1, -2)
(0, 3, 1, -1)
(0, 3, 1, 2)
(0, 3, 2, -3)
(0, 3, 2, -2)
(0, 3, 2, -1)
(0, 3, 2, 1)
(1, -3, -2, -1)
(1, -3, -2, 0)
(1, -3, -2, 2)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

(1, -3, -2, 3)
sigma after cycle: 5.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 12, 5.0)
(1, -3, -1, -2)
(1, -3, -1, 0)
(1, -3, -1, 2)
sigma after cycle: 3.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

(1, -3, -1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going t

...

(3, -2, 2, -3)
(3, -2, 2, -1)
(3, -2, 2, 0)
(3, -2, 2, 1)
sigma after cycle: 3.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 3.0)
(3, -1, -3, -2)
(3, -1, -3, 0)
(3, -1, -3, 1)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 10, 2.0)
(3, -1, -3, 2)
sigma after cycle: 3.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 11, 3.0)
(3, -1, -2, -3)
(3, -1, -2, 0)
(3, -1, -2, 1)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 2.0)
(3, -1, -2, 2)
sigma after cycle: 3.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.62962963]]), 4, 2, 9, 3.0)
(3, -1, 0, -3)
(3, -1, 0, -2)
(3, -1, 0,

(matrix([[ 0.77777778]]), 4, 2, 9, 4.0)
(-3, -1, 1, -2)
(-3, -1, 1, 0)
(-3, -1, 1, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-2.00e+00]
[ 2.00e+00]

The next result is going to be added: [0][0]
(matrix([[  1.23382207e-16]]), 4, 2, 9, 2.0)
(-3, -1, 1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-2.00e+00]
[ 2.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.77777778]]), 4, 2, 9, 4.0)
(-3, -1, 2, -2)
(-3, -1, 2, 0)
(-3, -1, 2, 1)
running... let's kill it...
(-3, -1, 2, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-3.00e+00]
[ 3.00e+00]

The next result is going to be added: [0][0]
(matrix([[  9.84455573e-17]]), 4, 2, 9, 4.0)
(-3, -1, 3, -2)
(-3, -1, 3, 0)
(-3, -1, 3, 1)
running... let's kill it...
(-3, -1, 3, 2)
running... let's kill it...
(-3, 0, -2, -1)
(-3, 0, -2, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 9, 1.666666666666666

(-2, -3, 2, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-3.00e+00]
[ 3.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 15, 4.0)
(-2, -3, 3, -1)
(-2, -3, 3, 0)
(-2, -3, 3, 1)
running... let's kill it...
(-2, -3, 3, 2)
running... let's kill it...
(-2, -1, -3, 0)
(-2, -1, -3, 1)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 0, 12, 2.0)
(-2, -1, -3, 2)
sigma after cycle: 3.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 15, 3.0)
(-2, -1, -3, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[-1.00e+00]
[ 0.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 17, 4.0)
(-2, -1, 0, -3)
(-2, -1, 0, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.22222222]]), 4, 1,

(matrix([[ 0.]]), 4, 2, 15, 5.0)
(-1, -3, 0, -2)
(-1, -3, 0, 1)
sigma after cycle: 1.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 1, 11, 1.6666666666666667)
(-1, -3, 0, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 13, 2.0)
(-1, -3, 0, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-1.00e+00]
[ 1.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 15, 4.0)
(-1, -3, 1, -2)
(-1, -3, 1, 0)
(-1, -3, 1, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-2.00e+00]
[ 2.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 13, 2.0)
(-1, -3, 1, 3)
sigma after cycle: 4.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-2.00e+00]
[ 2.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.]]), 4, 2, 15, 4.0)
(-1, -3, 2, -2)
(-1, -3, 2, 0)
(-1, -3, 2, 1)
running..

sol: [ 0.00e+00]
[-1.00e+00]
[-1.00e+00]
[ 2.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.44444444]]), 4, 2, 6, 2.0)
(-1, 3, 1, -3)
(-1, 3, 1, -2)
(-1, 3, 1, 0)
(-1, 3, 1, 2)
sigma after cycle: 2.0
sol: [ 0.00e+00]
[ 0.00e+00]
[-2.00e+00]
[ 2.00e+00]

The next result is going to be added: [0][0]
(matrix([[ 0.44444444]]), 4, 2, 5, 2.0)
(-1, 3, 2, -3)
(-1, 3, 2, -2)
(-1, 3, 2, 0)
(-1, 3, 2, 1)
